# Regresja liniowa wielu zmiennych

Stwórz model regresji liniowej dla cen mieszkań w Poznaniu. Zestaw danych to 5000 ogłoszeń z serwisu gratka.pl.
Miarą błędu na danych testowych niech będzie średnia kwadratowa błędu (Mean Squared Error, RMSE) dla cen mieszkań.
Plik gratkapl.csv zawiera dane, na których należy zbudować model regresyjny. Pierwszy wiersz w pliku to nagłówek zawierający opisy kolumn:
Id - numer porządkowy oferty
Expected - cena mieszkania
Rooms - liczba pokojów
SqrMeters - metraż mieszkania w m2
Floor - piętro
Location - dzielnica lub adres w Poznaniu
Description - fragment opisu mieszkania

Podziel dane źródłowe na dwa zbiory:
zbiór uczący, zawierający 4000 przykładów
zbiór testowy, zawierajacy 1000 przykładów
Pamiętaj, że dane są „surowe”, więc mogą zawierać obserwacje odstające (zob. wykład 5) – dobrze by było je usunąć przed przystąpieniem do budowy modelu.
Napisz algorytm, który przewidzi cenę mieszkania na podstawie trzech spośród powyższych cech (liczba pokojów, metraż, piętro) – użyj w tym celu regresji liniowej wielu zmiennych. Wytrenuj algorytm na danych ze zbioru uczącego.
Przetestuj algorytm na danych ze zbioru testowego, obliczając błąd średniokwadratowy (MSE) między cenami mieszkań ze zbioru testowego a cenami mieszkań obliczonymi na podstawie danych ze zbioru testowego.
Oprócz oczywistych cech numerycznych jak metraż, cechy mogą być symbolicznymi cechami opisującymi występowanie wyrazów lub kombinacjami kilku cech. Zaprojektuj w ten sposób co najmniej trzy nowe cechy i wykorzystaj je do budowy modelu regresji liniowej wielu zmiennych w połączeniu z cechami, których użyłeś w części podstawowej tego zadania.
Porównaj uzyskany błąd średniokwadratowy na zbiorze testowym z błędem obliczonym w części podstawowej tego zadania. Wykorzystaj gotowy model z pakietu scikit-learn.

## Importy

In [13]:
from sklearn import preprocessing 
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np
import pandas as pd
from scipy import stats

df_raw=pd.read_csv("gratkapl.csv")
df=df_raw

## Zapoznanie danych

In [14]:
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Id           5000 non-null   int64  
 1   Expected     5000 non-null   int64  
 2   Rooms        5000 non-null   int64  
 3   SqrMeters    5000 non-null   float64
 4   Floor        5000 non-null   int64  
 5   Location     5000 non-null   object 
 6   Description  5000 non-null   object 
dtypes: float64(1), int64(4), object(2)
memory usage: 273.6+ KB


In [15]:
df['Expected'].sort_values()

971            1
4528        4000
1175       68000
4889       79000
470        85000
          ...   
3620     3740000
4878    29979840
1173    29979840
1314    30542880
745     36378404
Name: Expected, Length: 5000, dtype: int64

In [5]:
df['Rooms'].sort_values()

4999     1
3551     1
493      1
4216     1
2900     1
        ..
3786     6
2099     6
2289     6
4058     7
3845    10
Name: Rooms, Length: 5000, dtype: int64

In [6]:
df['SqrMeters'].sort_values()

1341     14.37
2760     17.92
3396     18.00
2318     18.00
3234     18.00
         ...  
3629    190.00
939     192.40
188     193.00
3620    219.78
1424    257.00
Name: SqrMeters, Length: 5000, dtype: float64

In [7]:
df['Floor'].sort_values()

0        1
2519     1
2524     1
2525     1
2530     1
        ..
1687    16
3556    16
2936    16
308     16
500     16
Name: Floor, Length: 5000, dtype: int64

## Preprocessing i model

In [16]:
#usuniecie wartosci odstajacych
df_brute=df[(np.abs(stats.zscore(df['Expected'])) < 4)]
df_brute['Expected'].sort_values()
df=df_brute.drop([971])
df=df.drop([4528])
df=pd.DataFrame(df[['Expected', 'Rooms', 'SqrMeters', 'Floor']])
#skalowanie metoda niewrazliwa na wartosci odstajace
r_scaler = preprocessing.RobustScaler().fit(df)
df_r_scaled = r_scaler.transform(df)
#normalizacja
normalizer2=preprocessing.Normalizer().fit(df_r_scaled)
dfr_normalized=normalizer2.transform(df_r_scaled)
#podzial danych 
split_point = int(0.8 * len(dfr_normalized))
df_train2 = dfr_normalized[:split_point]
df_test2 = dfr_normalized[split_point:]
#uczenie
y_train2 = df_train2[:,0] #dane treningowe
x_train2 = df_train2[:,1:]
y_test2 = df_test2[:,0] #dane testowe
x_test2 = df_test2[:,1:]

model2 = LinearRegression() #definicja modelu
model2.fit(x_train2, y_train2) #dopasowanie modelu do danych

#predykcja wynikow na podstawie opracowanego modelu
y_predicted2 = model2.predict(x_test2)
print(y_predicted2[:10])
#ewaluacja - blad sredniokwadratowy
mse2 = mean_squared_error(y_predicted2, y_test2)
print("Błąd średniokwadratowy wynosi ", mse2)
print("Score: ",model2.score(x_test2,y_test2))

[-0.69554877  0.19193977 -0.32211543 -0.31147314  0.12113514 -0.08950423
 -0.59464225  0.29291885 -0.11736846 -0.57740091]
Błąd średniokwadratowy wynosi  0.16179757359957614
Score:  0.40272596267831595


## Czesc 2

In [11]:
Dzielnice = ['Antoninek','Zieliniec','Kobypole','Chartowo','Fabianowo','Kotowo','Główna','Głuszyna','Górczyn','Junikowo','Krzesiny','Pokrzywno','Garaszewo','Kwiatowe','Ławica','Ogrody','Naramowice','Sołacz','Rataje','Stare Miasto','Strzeszyn','Świerczewo','Wilda','Winiary','Wola','Żegrze','Jeżyce','Grunwald','Nowe Miasto', 'Piątkowo', 'Wilda', 'Podolany', 'Górczyn', 'Łazarz','Kiekrz'
            ,'Strzeszyn','Umultowo','Naramowice','Morasko']
Pozytywny_opis = ["duże", "duży", "okazja", "przestronne"]
Cechy_poprzednie=['Expected','Rooms','SqrMeters','Floor']

data=df_raw
for i in Dzielnice:
    data[i]=data['Location'].str.contains(i, case=False, flags=0, na=None, regex=True) #Dodanie kolumn-dzielnic i okresienie boolowskie czy lokalizacja lezy w danej dzielnicy
    data[i]=data[i].apply(lambda x: 1 if x == True else 0) #zamiana typu bool na int
for i in Pozytywny_opis:
    data[i]=data['Description'].str.contains(i, case=False, flags=0, na=None, regex=True)  #Dodanie kolumn-pozytywne-slowa-z-opisu i okresienie boolowskie czy te pozytywne slowa sa w opisie dzielnicy
    data[i]=data[i].apply(lambda x: 1 if x == True else 0) #zamiana typu bool na int
data = data[Cechy_poprzednie+Dzielnice+Pozytywny_opis]

#preprocessing
#skalowanie metoda niewrazliwa na wartosci odstajace
r_scaler = preprocessing.RobustScaler().fit(data)
data_r_scaled = r_scaler.transform(data)
#normalizacja
normalizer2=preprocessing.Normalizer().fit(data_r_scaled)
dfr_normalized=normalizer2.transform(data_r_scaled)
#podzial danych 
split_point = int(0.8 * len(dfr_normalized))
df_train2 = dfr_normalized[:split_point]
df_test2 = dfr_normalized[split_point:]
#uczenie
y_train2 = df_train2[:,0] #dane treningowe
x_train2 = df_train2[:,1:]
y_test2 = df_test2[:,0] #dane testowe
x_test2 = df_test2[:,1:]
model2 = LinearRegression() #definicja modelu
model2.fit(x_train2, y_train2) #dopasowanie modelu do danych

#predykcja wynikow na podstawie opracowanego modelu
y_predicted2 = model2.predict(x_test2)
print(y_predicted2[:10])
#ewaluacja - blad sredniokwadratowy
mse2 = mean_squared_error(y_predicted2, y_test2)
print("Błąd średniokwadratowy wynosi ", mse2)
print("Score: ",model2.score(x_test2,y_test2))

[ 0.19145291 -0.33719549 -0.22518117  0.1248904   0.09745053 -0.61960586
  0.32463771 -0.10097765 -0.60111697 -0.2159007 ]
Błąd średniokwadratowy wynosi  0.08644623838274855
Score:  0.4867592659663338
